In [28]:
import redis
import pandas as pd
from datetime import datetime

db = redis.Redis(host='127.0.0.1', port=6379, db=0, charset="utf-8", decode_responses=True)
db.flushdb()

True

In [27]:
sobrante=pd.read_csv('C:/Users/Pabul/Downloads/productosTest.csv')
gastosG= pd.read_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/data/GastosExtra.csv')
ingresosi= pd.read_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/data/IngresosExtra.csv')
ingresosi

,Fecha,Concepto,Ingreso,Usuario
0,2022-06-05 20:08:36,Prepago,50,HPPS125982733
1,2022-06-05 20:09:54,Devoluciones,33,HPPS125982733
2,2022-06-05 20:10:01,Prepago,7,HPPS125982733
3,2022-06-05 20:10:01,Prepago,31,HPPS125982733
4,2022-06-05 20:10:11,Prepago,73,HSDP02837162
...,...,...,...,...
98,2022-06-05 20:14:48,Devoluciones,61,MMCL10294873
99,2022-06-05 20:14:48,Anticipo,76,HSDP02837162
100,2022-06-05 20:14:49,Emergencias,20,MMCL10294873
101,2022-06-05 20:14:50,Devoluciones,41,HPPS125982733


In [26]:

def carga_datosPrueba(pruebasG, pruebasI):
    key1G='gastoExtra:Gasto:'
    key2G='gastoExtra:Concepto:'
    key3G='gastoExtra:Hora:'
    key4G='gastoExtra:Usuario:'

    key1I='ingresoExtra:Ingreso:'
    key2I='ingresoExtra:Concepto:'
    key3I='ingresoExtra:Hora:'
    key4I='ingresoExtra:Usuario:'
    for index, row in pruebasG.iterrows():
        db.rpush(key1G, row[2])
        db.rpush(key2G, row[1])
        db.rpush(key3G, row[0])
        db.rpush(key4G, row[3])
        
    for index, row in pruebasI.iterrows():
        db.rpush(key1I, row[2])
        db.rpush(key2I, row[1])
        db.rpush(key3I, row[0])
        db.rpush(key4I, row[3])
    

def carga_productos(restantes):
    for index, row in sobrante.iterrows():
        key= 'Productos:'+row[0]
        result= db.set(key, int(row[1]))
    return result

def get_sobrantes(producto):
    key="Productos:"+producto
    result = db.get(key)
    return int(result)

def producidos(producto, producido):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)+producido)
    return result

def comprados(producto, comprado):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)-comprado)
    return result

def gastosExtra(gastado, concepto, user):
    hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key1='gastoExtra:Gasto:'
    key2='gastoExtra:Concepto:'
    key3='gastoExtra:Hora:'
    key4='gastoExtra:Usuario:'
    db.rpush(key1, gastado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return 

def gastosExtraTotales():
    gastos= db.lrange('gastoExtra:Gasto:', 0, -1)
    return sum(map(int, gastos))

def ingresosExtra(ingresado, concepto, user):
    hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key1='ingresoExtra:Ingreso:'
    key2='ingresoExtra:Concepto:'
    key3='ingresoExtra:Hora:'
    key4='ingresoExtra:Usuario:'
    db.rpush(key1, ingresado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return

def dfIngresosExtra(self):
        tiempos=[]
        for i in self.db.lrange('ingresoExtra:Hora:', 0, -1):
            tiempos.append(datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))
        ingdict={'Fecha': tiempos, 'Concepto':self.db.lrange('ingresoExtra:Concepto:', 0, -1), \
            'Ingreso': self.db.lrange('ingresoExtra:Ingreso:', 0, -1), 'Usuario': self.db.lrange('ingresoExtra:Usuario:', 0, -1)}
        ingdf=pd.DataFrame.from_dict(data=ingdict)
        return( ingdf)

def dfGastosExtra():
        tiempos=[]
        for i in db.lrange('gastoExtra:Hora:', 0, -1):
            tiempos.append(datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))
        gastdict= {'Fecha': tiempos, 'Concepto':db.lrange('gastoExtra:Concepto:', 0, -1), \
            'Costo': db.lrange('gastoExtra:Gasto:', 0, -1), 'Usuario': db.lrange('gastoExtra:Usuario:', 0, -1)}
        gastodf=pd.DataFrame.from_dict(data=gastdict)
        return(gastodf)


carga_productos(sobrante)
print(get_sobrantes('Volovan'))
producidos('Volovan', 15)
print(get_sobrantes('Volovan'))
comprados('Volovan', 1)
print(get_sobrantes('Volovan'))

gastosExtra(1, 'Manzanas', 'Yeu')
gastosExtra(17, 'Peras', 'Pablo')

ingresosExtra(91, 'Anticipo', 'Marcela')
ingresosExtra(17, 'Prepago', 'Pablo')
ingresosExtra(94, 'Prepago', 'Yeu')

print(gastosExtraTotales())

print(db.lrange('gastoExtra:Gasto:', 0, -1))
print(db.lrange('gastoExtra:Concepto:', 0, -1))
print(db.lrange('gastoExtra:Hora:', 0, -1))
print(db.lrange('gastoExtra:Usuario:', 0, -1))

print(db.lrange('ingresoExtra:Ingreso:', 0, -1))
print(db.lrange('ingresoExtra:Concepto:', 0, -1))
print(db.lrange('ingresoExtra:Hora:', 0, -1))
print(db.lrange('ingresoExtra:Usuario:', 0, -1))

10
25
24
4570
['81', '22', '59', '71', '18', '12', '94', '14', '89', '97', '10', '28', '70', '96', '33', '61', '80', '14', '40', '29', '10', '1', '34', '67', '73', '68', '90', '48', '45', '29', '22', '66', '86', '34', '13', '31', '96', '11', '39', '45', '44', '21', '66', '74', '87', '63', '14', '85', '49', '69', '44', '28', '54', '82', '79', '46', '87', '76', '43', '27', '87', '10', '73', '28', '15', '40', '66', '3', '27', '51', '32', '57', '18', '85', '91', '19', '10', '6', '50', '85', '29', '13', '1', '14', '33', '46', '72', '67', '8', '27', '7', '86', '2', '6', '95', '28', '68', '31', '2', '1', '17']
['Detergente', 'Manzanas', 'Manzanas', 'Manzanas', 'Charloa', 'Aceite', 'Charloa', 'Aceite', 'Quita Grasa', 'Quita Grasa', 'Quita Grasa', 'Quita Grasa', 'Aceite', 'Detergente', 'Aceite', 'Manzanas', 'Charloa', 'Aceite', 'Aceite', 'Detergente', 'Quita Grasa', 'Aceite', 'Detergente', 'Manzanas', 'Quita Grasa', 'Aceite', 'Detergente', 'Detergente', 'Charloa', 'Manzanas', 'Detergente', 'Det

In [74]:
datos=cerrarDia()
datos[0]

,Fecha,Concepto,Costo,Usuario
0,05/06/2022 14:37:21,Detergente,65,Yeu
1,05/06/2022 14:37:27,Quita Grasa,47,ProfePime
2,05/06/2022 14:37:30,Manzanas,71,Yeu
3,05/06/2022 14:37:30,Manzanas,35,Marcela
4,05/06/2022 14:37:38,Aceite,12,ProfePime
...,...,...,...,...
88,05/06/2022 14:42:08,Charloa,39,Pablo
89,05/06/2022 14:42:08,Detergente,14,ProfePime
90,05/06/2022 14:42:11,Manzanas,59,Yeu
91,05/06/2022 14:42:12,Aceite,8,Marcela


In [75]:
datos[1]

,Fecha,Concepto,Ingreso,Usuario
0,05/06/2022 14:37:19,Emergencias,7,Pablo
1,05/06/2022 14:37:20,Devoluciones,37,Yeu
2,05/06/2022 14:37:23,Emergencias,42,ProfePime
3,05/06/2022 14:37:24,Devoluciones,28,Marcela
4,05/06/2022 14:37:27,Emergencias,68,Marcela
...,...,...,...,...
102,05/06/2022 14:41:56,Prepago,70,Yeu
103,05/06/2022 14:42:01,Anticipo,67,Pablo
104,05/06/2022 14:42:04,Devoluciones,24,Yeu
105,05/06/2022 14:42:07,Anticipo,53,Yeu


In [73]:
"""import time
import numpy as np
#Generando datos
gconcept=['Manzanas', 'Aceite', 'Quita Grasa', 'Detergente', 'Charloa']
users=['Yeu', 'Pablo', 'Marcela', 'ProfePime']
ingconcept=['Anticipo', 'Devoluciones', 'Prepago', 'Emergencias']

for i in range(0,200):
    if(np.random.randint(2)==1):
        ingresosExtra(np.random.randint(100), ingconcept[np.random.randint(4)],users[np.random.randint(4)])
    else:
        gastosExtra(np.random.randint(100), gconcept[np.random.randint(5)],users[np.random.randint(4)])
    time.sleep(np.random.randint(4))
    """

In [76]:
datos[0].to_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/CostosExtra.csv', index=False)
datos[1].to_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/IngresosExtra.csv' , index=False)

In [29]:
def carga_datosPrueba(pruebasG, pruebasI):
    key1G='gastoExtra:Gasto:'
    key2G='gastoExtra:Concepto:'
    key3G='gastoExtra:Hora:'
    key4G='gastoExtra:Usuario:'

    key1I='ingresoExtra:Ingreso:'
    key2I='ingresoExtra:Concepto:'
    key3I='ingresoExtra:Hora:'
    key4I='ingresoExtra:Usuario:'
    for index, row in pruebasG.iterrows():
        db.rpush(key1G, row[2])
        db.rpush(key2G, row[1])
        db.rpush(key3G, row[0])
        db.rpush(key4G, row[3])
        
    for index, row in pruebasI.iterrows():
        db.rpush(key1I, row[2])
        db.rpush(key2I, row[1])
        db.rpush(key3I, row[0])
        db.rpush(key4I, row[3])
    
carga_datosPrueba(gastosG, ingresosi)

In [30]:
k=dfGastosExtra()
k

,Fecha,Concepto,Costo,Usuario
0,2022-06-05 20:08:37,Detergente,81,MTYSJ12476412
1,2022-06-05 20:09:55,Manzanas,22,MTYSJ12476412
2,2022-06-05 20:09:57,Manzanas,59,HSDP02837162
3,2022-06-05 20:10:00,Manzanas,71,MMCL10294873
4,2022-06-05 20:10:03,Charloa,18,HPPS125982733
...,...,...,...,...
94,2022-06-05 20:14:45,Aceite,95,HPPS125982733
95,2022-06-05 20:14:49,Manzanas,28,MMCL10294873
96,2022-06-05 20:14:50,Manzanas,68,MTYSJ12476412
97,2022-06-05 20:14:50,Quita Grasa,31,MMCL10294873
